In [1]:
# 동적 사이트 수집
from selenium import webdriver
# 크롬 드라이버 설치
from webdriver_manager.chrome import ChromeDriverManager
# 자동적 접근
from selenium.webdriver.chrome.service import Service 
# 크롭 드라이버 옵션 지정
from selenium.webdriver.chrome.options import Options
# find_element 함수 쉽게 쓰기 위함 
from selenium.webdriver.common.by import By 
from selenium.webdriver.common.keys import Keys
import time
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [9]:
l_dt = 1.5
s_dt = 0.5
count = 0

def check_add(word):
    w_L = []
    for w in word.split():
        if w not in w_L:
            w_L.append(w)
    return ' '.join(w_L)

csv_list = ['Seoul.csv','Busan.csv','Daegu.csv','Incheon.csv','Gwangju.csv','Daejeon.csv','Ulsan.csv','Gyeonggi.csv','Gangwon.csv',\
    'Chung_buk.csv','Chung_nam.csv','Jeollabuk.csv','Jeollanam.csv','Gyeongsangbuk.csv','Gyeongsangnam.csv','Jeju.csv','Sejong.csv']
result_L = []

url = 'https://map.naver.com/p?c=15.00,0,0,0,dh'
driver = webdriver.Chrome() 
driver.get(url)
time.sleep(l_dt)

search_btn = driver.find_element(By.CSS_SELECTOR, 'div.input_box > input.input_search')
search_btn.click()
time.sleep(s_dt)

for csv_i in range(len(csv_list)):
    
    df = pd.read_csv(f'C:/python/mini_project_1/{csv_list[csv_i]}')
    df.columns = ('0','업종','업소명','주요품목','가격','연락처','주소')
    df.drop('0',axis=1,inplace=True)
    
    df['수정주소'] =  df['주소'].apply(lambda x : check_add(x))
    df['검색어'] = df['수정주소'].apply(lambda x : x.split()[0]) + ' ' + df['수정주소'].apply(lambda x : x.split()[1]) + ' ' + df['업소명']
    
    for k_word in df['검색어'].values:
        
        count = count + 1
        
        if count == 500:
            print("인터넷새창열기")
            url = 'https://map.naver.com/p?c=15.00,0,0,0,dh'
            driver = webdriver.Chrome() 
            driver.get(url)
            time.sleep(l_dt)

            search_btn = driver.find_element(By.CSS_SELECTOR, 'div.input_box > input.input_search')
            search_btn.click()
            time.sleep(s_dt)
            count = 0

        search_btn.send_keys(k_word)
        search_btn.send_keys(Keys.ENTER)
        time.sleep(l_dt)
        
        try:
            driver.switch_to.frame('searchIframe')
            try:
                #print("선택1")
                item_btn = driver.find_element(By.CSS_SELECTOR,'#app-root > div > div > div > div:nth-child(2) > div:nth-child(1) > div:nth-child(2) > span')
                #집에서 됨 item_btn = driver.find_element(By.CSS_SELECTOR,'#app-root > div > div > divnth-child(1) > ul > li:nth-child(1) > div:nth-child(1) > div:nth-child(2) > a')
                #item_btn = driver.find_element(By.XPATH,'/html/body/div[3]/div/div[2]/div[1]/ul/li[1]/div[1]')
                item_btn.click()
            except:
                try:
                    #print("선택2")
                    item_btn = driver.find_element(By.XPATH,'//*[@id="_pcmap_list_scroll_container"]/ul/li/div[1]/a[1]')
                    item_btn.click()
                except:
                    try:
                        #print("선택3")
                        item_btn = driver.find_element(By.CSS_SELECTOR,'#_pcmap_list_scroll_container > ul > li:nth-child(1) > div.qbGlu > div.ouxiq > a:nth-child(1) > div > div')
                        item_btn.click()
                    except:
                        #print("선택4")
                        item_btn = driver.find_element(By.XPATH,'/html/body/div[3]/div/div[2]/div[1]/ul/li[1]/div[1]')
                        item_btn.click()
                        
            time.sleep(l_dt)
            
        except:
            pass
        
        try:
            #print("세부선택")
            driver.switch_to.default_content()    
            driver.switch_to.frame('entryIframe')
#           time.sleep(s_dt)
        except:
            pass

        try:
            #item = driver.find_element(By.XPATH,'//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]')
            item = driver.find_element(By.XPATH,'//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]/span[1]')
            result_L.append(item.text)
            print(csv_i, k_word)
            print(result_L[-1])
            print("-"*20)   
            
        except:
            result_L.append('별점없음')
            print(result_L[-1])
            print("-"*20)
        
        driver.switch_to.default_content()
#        time.sleep(s_dt)
        
        c_btn = driver.find_element(By.XPATH,'//*[@id="app-layout"]/div[1]/div/div[1]/div/div/button[2]')
        c_btn.click()
#        time.sleep(s_dt)
    
    df['별점'] = result_L
    df.to_csv(f'C:/python/mini_project_1/ver2_{csv_list[csv_i]}')
    result_L.clear()

0 서울특별시 동대문구 신원식당
별점
4.78
--------------------
0 서울특별시 은평구 신미가 맛집
방문자리뷰 32
--------------------
0 서울특별시 은평구 북한산 손칼국수
별점
4.32
--------------------
0 서울특별시 은평구 응암칼국수
별점
4.35
--------------------
0 서울특별시 은평구 따뜻한 밥상
방문자리뷰 280
--------------------
0 서울특별시 송파구 새서울세탁
블로그리뷰 1
--------------------
0 서울특별시 송파구 송천엄마손칼국수
방문자리뷰 43
--------------------
0 서울특별시 송파구 방이뚝배기
방문자리뷰 1
--------------------
별점없음
--------------------
별점없음
--------------------
별점없음
--------------------
0 서울특별시 동대문구 서희
별점
4.36
--------------------
0 서울특별시 동대문구 신라방마라탕
별점
4.44
--------------------
0 서울특별시 동대문구 만나식당
별점
4.16
--------------------
0 서울특별시 동대문구 오뚝식당
별점
4.49
--------------------
0 서울특별시 동대문구 이야기카페
별점
4.42
--------------------
별점없음
--------------------


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=120.0.6099.110)
Stacktrace:
	GetHandleVerifier [0x00007FF717E72142+3514994]
	(No symbol) [0x00007FF717A90CE2]
	(No symbol) [0x00007FF7179376AA]
	(No symbol) [0x00007FF717910AFD]
	(No symbol) [0x00007FF7179ACB1B]
	(No symbol) [0x00007FF7179C218F]
	(No symbol) [0x00007FF7179A5D93]
	(No symbol) [0x00007FF717974BDC]
	(No symbol) [0x00007FF717975C64]
	GetHandleVerifier [0x00007FF717E9E16B+3695259]
	GetHandleVerifier [0x00007FF717EF6737+4057191]
	GetHandleVerifier [0x00007FF717EEE4E3+4023827]
	GetHandleVerifier [0x00007FF717BC04F9+689705]
	(No symbol) [0x00007FF717A9C048]
	(No symbol) [0x00007FF717A98044]
	(No symbol) [0x00007FF717A981C9]
	(No symbol) [0x00007FF717A888C4]
	BaseThreadInitThunk [0x00007FF8D3FF7614+20]
	RtlUserThreadStart [0x00007FF8D57426B1+33]
